In [33]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv
from langchain_gigachat.chat_models import GigaChat

load_dotenv(find_dotenv())


model = GigaChat(
    model='GigaChat-2',
    verify_ssl_certs=False,
    max_tokens=2500,
)

In [34]:
database = {
    "schema": "crm",
    "tables": {
        "customers": {
            "description": "Содержит общую информацию о клиентах.",
            "columns": {
                "customer_id": "Целочисленный идентификатор клиента.",
                "first_name": "Имя клиента (строка).",
                "last_name": "Фамилия клиента (строка).",
                "birth_year": "Год рождения клиента (целое число).",
                "registration_year": "Год регистрации в системе (целое число)."
            }
        },
        "contacts": {
            "description": "Контактная информация клиентов.",
            "columns": {
                "contact_id": "Целочисленный идентификатор записи контакта.",
                "customer_id": "Целочисленный ID клиента, к которому относится контакт.",
                "email": "Электронная почта (строка).",
                "phone": "Номер телефона (строка)."
            }
        },
        "purchases": {
            "description": "Информация о покупках клиентов.",
            "columns": {
                "purchase_id": "Целочисленный идентификатор записи о покупке.",
                "customer_id": "Целочисленный ID клиента, который совершил покупку.",
                "purchase_year": "Год покупки (целое число).",
                "amount_rub": "Сумма покупки в рублях (целое число).",
                "product_type": "Тип продукта (строка)."
            }
        }
    }
}


In [35]:
from typing import Dict

from langchain.tools import tool


@tool
def get_all_table_descriptions() -> str:
    """Возвращает названия всех таблиц в базе данных и их описание"""
    print("\033[92m" + "Бот запросил информацию о таблицах" + "\033[0m")
    all_tables = database['tables']
    return " | ".join([f'Таблица: {table_name}, Описание: {table_info["description"]}' for table_name, table_info in all_tables.items()])


@tool
def get_table_info(table_name: str) -> Dict:
    """Возвращает список полей и их описание для указанной таблицы"""
    print("\033[92m" + "Бот запросил информацию о списке полей в таблице" + "\033[0m")
    if table_name not in database['tables']:
        raise ValueError(f"Таблица '{table_name}' не найдена в базе данных.")
    table_info = database['tables'][table_name]
    return table_info

In [44]:
system_prompt = """
Ты - помощник-аналитик, который отвечает на вопросы о базе данных (postgres), содержащую информацию о клиентах.
Ты оперируешь языком программирования SQL и в зависимости от вопроса, можешь использовать инструменты для получения информации о базе данных.
Ты умеешь писать оптимизированные SQL-запросы, которые быстро отрабатывают на больших объемах данных.

Ты должен уточнять всю необходимую информацию у пользователя и при помощи инструментов. 
Тебе даны инструменты, которые помогут получить информацию о базе данных:
1. get_all_table_descriptions - возвращает названия всех таблиц в базе данных и их описание.
2. get_table_info - возвращает список полей и их описание для указанной таблицы.
Ты должен использовать эти инструменты, чтобы получить всю необходимую информацию для ответа на вопрос пользователя.

Получив необходимую информацию, тебе нужно сформировать SQL-запрос. 
Сначала необходимо спланировать формирование запроса, прописать, какие таблицы и поля тебе необходимы и зачем. Затем сформируй SQL-запрос (postgres).
ВАЖНО! В ответ на вопрос ты должен предоставить только SQL-запрос, без каких-либо пояснений или дополнительной информации.
ВАЖНО! Прежде чем отправить ответ, обязательно проверь, что запрос корректен и не содержит ошибок.
"""

In [45]:
tools = [get_all_table_descriptions, get_table_info]

In [46]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model,
                           tools=tools,
                           prompt=system_prompt)

Пример запроса:

Для всех клиентов 2000 года рождения выведи среднюю сумму покупок для каждого типа продукта. Отсортируй по продукту

In [47]:
while(True):
    rq = input("\Пользователь: ")
    print("Пользователь: ", rq)
    if rq == "":
        break
    resp = agent.invoke({"messages": [("user", rq)]})
    print("Ассистент: ", resp["messages"][-1].content)

Пользователь:  Для всех клиентов 2000 года рождения выведи среднюю сумму покупок для каждого типа продукта. Отсортируй по продукту
Бот запросил информацию о таблицах
Бот запросил информацию о списке полей в таблице
Бот запросил информацию о списке полей в таблице
Ассистент:  ```sql
SELECT product_type, AVG(amount_rub) as avg_purchase_amount
FROM purchases
JOIN customers ON purchases.customer_id = customers.customer_id
WHERE customers.birth_year = 2000
GROUP BY product_type
ORDER BY product_type;
```
Пользователь:  Выведи суммарный объем продаж в течение всех лет, отсортируй по годам
Бот запросил информацию о таблицах
Бот запросил информацию о списке полей в таблице
Ассистент:  SELECT purchase_year, SUM(amount_rub) AS total_sales
FROM purchases
GROUP BY purchase_year
ORDER BY purchase_year;
Пользователь:  
